# Machine Learning Assignment Week 3 Task 1

### Transfering Data in Padas dataframe using BeautifulSoup

###### Installing BeautifulSoup Library

In [16]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


###### importing required libraries

In [17]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [23]:
#Fetching data from Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [29]:
soup = BeautifulSoup(source, 'html.parser')

In [25]:
# Converting data into pandas dataframe

data = soup.find('table', class_='wikitable sortable').tbody
# print(data)

columns = ['PostCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)

for row in data.find_all('tr'):
    row_data = []
    for col in row.find_all('td'):
        row_data.append(col.text)
        
    if row_data:
        df = pd.DataFrame({"PostCode" : [row_data[0]],
                      "Borough" : [row_data[1]],
                      "Neighborhood" : [row_data[2]]})
        toronto_data = toronto_data.append(df, ignore_index=True)
toronto_data.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [30]:
# Cleaning the data
toronto_data['Neighborhood'] = [value.split('\n')[0] for value in toronto_data['Neighborhood']]

# Removing all rows having 'Not assigned' value to both Neighborhood and Borough column
toronto_data = toronto_data[(toronto_data.Neighborhood != 'Not assigned') | (toronto_data.Borough != 'Not assigned')]

# Assigning Borough to Neighborhood columns having values as 'Not assigned'
toronto_data.loc[toronto_data.Neighborhood == 'Not assigned', 'Neighborhood'] = toronto_data['Borough']

In [31]:
# Merging rows having same 'PostCode' values

# postcodes = toronto_data['PostCode'].unique()
# print(postcodes.shape)

toronto_data = toronto_data.groupby('PostCode', as_index=False).agg(','.join)
toronto_data['Borough'] = toronto_data['Borough'].str.split(',').str[0]

In [34]:
toronto_data.head(12)


,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"
